<a href="https://colab.research.google.com/github/LC1332/Prophet-Andrew-Ng/blob/main/langchain/LangChain%E9%A1%B9%E7%9B%AE%E5%88%86%E6%9E%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip -q install -U github-dependents-info

In [1]:
!wget https://python.langchain.com/docs/ecosystem/dependents

--2023-08-23 08:20:42--  https://python.langchain.com/docs/ecosystem/dependents
Resolving python.langchain.com (python.langchain.com)... 76.76.21.98, 76.76.21.22
Connecting to python.langchain.com (python.langchain.com)|76.76.21.98|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84884 (83K) [text/html]
Saving to: ‘dependents’

dependents          100%[===================>]  82.89K  --.-KB/s    in 0.01s   

2023-08-23 08:20:42 (5.94 MB/s) - ‘dependents’ saved [84884/84884]



对于网页https://python.langchain.com/docs/ecosystem/dependents

我想用python下载页面上的内容，并保存到/content/page.html

请用python为我实现

In [2]:
import requests

url = 'https://python.langchain.com/docs/ecosystem/dependents'

response = requests.get(url)
content = response.text

with open('/content/page.html', 'w') as f:
    f.write(content)

我要分析一个html文件/content/page.html

里面有一个两列的表格，有大量形如

```html
<tr><td align="left"><a href="https://github.com/openai/openai-cookbook" target="_blank" rel="noopener noreferrer">openai/openai-cookbook</a></td><td align="right">46276</td></tr>
<tr><td align="left"><a href="https://github.com/AntonOsika/gpt-engineer" target="_blank" rel="noopener noreferrer">AntonOsika/gpt-engineer</a></td><td align="right">41497</td></tr>
```

这样的字段，请为我实现一段python程序，

提取其中的github链接，将链接和后面的数字保存为两个list

如
```python
github_url = ['https://github.com/openai/openai-cookbook', 'https://github.com/AntonOsika/gpt-engineer']

star_nums = [46276,41497]
```

In [3]:
import re
from bs4 import BeautifulSoup

with open('/content/page.html', 'r') as f:
    content = f.read()

soup = BeautifulSoup(content, 'html.parser')

github_urls = []
star_nums = []

for tr in soup.find_all('tr'):
    tds = tr.find_all('td')

    if len(tds) == 2:
        url = tds[0].find('a')['href']
        github_urls.append(url)

        num = int(tds[1].text)
        star_nums.append(num)

print(github_urls)
print(star_nums)

['https://github.com/openai/openai-cookbook', 'https://github.com/AntonOsika/gpt-engineer', 'https://github.com/imartinez/privateGPT', 'https://github.com/LAION-AI/Open-Assistant', 'https://github.com/microsoft/TaskMatrix', 'https://github.com/hpcaitech/ColossalAI', 'https://github.com/streamlit/streamlit', 'https://github.com/reworkd/AgentGPT', 'https://github.com/OpenBB-finance/OpenBBTerminal', 'https://github.com/geekan/MetaGPT', 'https://github.com/jerryjliu/llama_index', 'https://github.com/StanGirard/quivr', 'https://github.com/openai/chatgpt-retrieval-plugin', 'https://github.com/mindsdb/mindsdb', 'https://github.com/cube-js/cube', 'https://github.com/PromtEngineer/localGPT', 'https://github.com/mlflow/mlflow', 'https://github.com/chatchat-space/Langchain-Chatchat', 'https://github.com/GaiZhenbiao/ChuanhuChatGPT', 'https://github.com/logspace-ai/langflow', 'https://github.com/openai/evals', 'https://github.com/airbytehq/airbyte', 'https://github.com/databrickslabs/dolly', 'https

我有一个list of string分别是很多github repo的地址

github_urls = ['https://github.com/openai/openai-cookbook', 'https://github.com/AntonOsika/gpt-engineer']

我有办法实现一段python程序

依次提取每个repo的README.md吗？

比如第一个repo对应的readme在

https://raw.githubusercontent.com/openai/openai-cookbook/main/README.md

其实应该可以直接通过wget获取，

注意，有时候可能是Readme.md或者readme.md

请为我获取所有的readme，重命名成项目的名字保存到/content/output

In [4]:
!pip install -q wget

  Preparing metadata (setup.py) ... done


In [5]:
!rm -rf /content/output

!mkdir output

In [7]:
import os
import re

import wget
from tqdm import tqdm

output_dir = '/content/output'

save_names = []

count = 0

for url in tqdm(github_urls):

    count = count + 1

    repo_name = re.search(r'github.com/(.+)', url).group(1)

    readme_filenames = ['README.md', 'Readme.md', 'readme.md']

    downloaded = False

    for filename in readme_filenames:

        readme_url = f'https://raw.githubusercontent.com/{repo_name}/main/{filename}'

        save_name = repo_name.replace('/', '_') + '.md'

        outfile = os.path.join(output_dir, save_name)

        try:
            wget.download(readme_url, out=outfile)
            downloaded = True
            save_names.append( outfile )
            break
        except:
            pass

    # readme_url = f'https://raw.githubusercontent.com/{repo_name}/main/README.md'

    # save_name = repo_name.replace('/', '_')

    # print(f'Downloaded {outfile}')

    # outfile = os.path.join(output_dir, save_name + '.md')

    # wget.download(readme_url, out=outfile)



100%|██████████| 354/354 [01:48<00:00,  3.27it/s]


现在我们来考虑每个readme调用openAI进行自动总结

这里我们要先提取readme里面的前3k个字符

In [ ]:
print(save_names[0])

/content/output/openai_openai-cookbook.md


save_names 是一个python的list of string

max_token = 2000

里面是很多文件，对于里面每个save_name对应的文件

使用utf8编码打开,使用\n进行split

然后初始化一个字符串txt_show为空

把文件中的内容逐行添加到txt_show中，直到添加的长度即将超过max_token则停止

请用python为我实现

In [ ]:
# import codecs

# max_token = 2000

# for star_num, url, save_name in zip(star_nums, github_urls, save_names):

#     with codecs.open(save_name, 'r', 'utf-8') as f:
#         lines = f.read().split('\n')

#     txt_show = ''

#     total_len = 0

#     for line in lines:
#         if total_len + len(line) < max_token:
#             txt_show += line + '\n'
#             total_len += len(line) + 1
#         else:
#             break

#     print(txt_show)

#     break


# OpenAI Cookbook

The OpenAI Cookbook shares example code for accomplishing common tasks with the [OpenAI API].

To run these examples, you'll need an OpenAI account and API key ([create a free account][api signup]).

Most code examples are written in Python, though the concepts can be applied in any language.

[![Open in GitHub Codespaces](https://github.com/codespaces/badge.svg)](https://github.com/codespaces/new?hide_repo_select=true&ref=main&repo=468576060&machine=basicLinux32gb&location=EastUs)

## Recently added/updated 🆕 ✨
- [How to fine-tune chat models](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb) [Aug 22, 2023]
- [How to evaluate abstractive summarization](examples/evaluation/How_to_eval_abstractive_summarization.ipynb) [Aug 16, 2023]
- [Whisper prompting guide](examples/Whisper_prompting_guide.ipynb) [June 27, 2023]
- [Question answering using a search API and re-ranking](https://github.com/openai/openai-cookbook/blob/main/

In [ ]:
!pip -q install openai tiktoken langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.5/75.5 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.6 MB/s eta 0:00:00


In [ ]:
import tiktoken

enc = tiktoken.get_encoding("cl100k_base")

In [ ]:
import codecs

max_token = 2000

txt_show = ''

for star_num, url, save_name in zip(star_nums, github_urls, save_names):

    with codecs.open(save_name, 'r', 'utf-8') as f:
        lines = f.read().split('\n')

    txt_show = ''

    total_len = 0

    for line in lines:

        current_len = len( enc.encode(line + '\n') )
        if total_len + current_len < max_token:
            txt_show += line + '\n'
            total_len += current_len
        else:
            break

    print(txt_show)

    break


# OpenAI Cookbook

The OpenAI Cookbook shares example code for accomplishing common tasks with the [OpenAI API].

To run these examples, you'll need an OpenAI account and API key ([create a free account][api signup]).

Most code examples are written in Python, though the concepts can be applied in any language.

[![Open in GitHub Codespaces](https://github.com/codespaces/badge.svg)](https://github.com/codespaces/new?hide_repo_select=true&ref=main&repo=468576060&machine=basicLinux32gb&location=EastUs)

## Recently added/updated 🆕 ✨
- [How to fine-tune chat models](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb) [Aug 22, 2023]
- [How to evaluate abstractive summarization](examples/evaluation/How_to_eval_abstractive_summarization.ipynb) [Aug 16, 2023]
- [Whisper prompting guide](examples/Whisper_prompting_guide.ipynb) [June 27, 2023]
- [Question answering using a search API and re-ranking](https://github.com/openai/openai-cookbook/blob/main/

In [ ]:
print(lines)

['random_string']


In [ ]:
print(star_nums)


[46276, 41497, 36296, 34861, 33906, 31654, 26571, 25819, 23180, 21968, 20204, 20142, 19215, 17580, 16003, 15134, 15027, 14024, 12020, 11599, 11509, 11493, 10531, 9955, 9081, 8201, 7754, 7348, 6950, 6858, 6300, 6193, 6026, 5641, 5448, 5365, 5352, 5192, 4993, 4831, 4824, 4783, 4779, 4752, 4452, 4286, 4167, 3952, 3887, 3636, 3480, 3445, 3397, 3366, 3335, 3316, 3270, 3266, 3176, 2999, 2932, 2816, 2803, 2679, 2673, 2492, 2486, 2450, 2448, 2255, 2216, 2198, 2177, 2144, 2092, 2060, 2039, 1992, 1949, 1915, 1783, 1761, 1627, 1509, 1499, 1476, 1471, 1392, 1370, 1360, 1357, 1345, 1342, 1332, 1314, 1314, 1313, 1299, 1237, 1232, 1223, 1192, 1126, 1117, 1110, 1096, 1080, 1075, 1068, 984, 957, 955, 944, 942, 909, 899, 896, 889, 868, 854, 847, 836, 818, 798, 782, 748, 741, 732, 722, 710, 710, 707, 704, 704, 692, 682, 670, 662, 650, 632, 624, 617, 602, 588, 585, 581, 569, 568, 559, 558, 554, 537, 534, 534, 524, 496, 495, 494, 492, 490, 488, 481, 480, 480, 473, 471, 467, 463, 463, 463, 441, 437, 432, 43

好了，接下来就是加一个总结

In [ ]:
import os
import openai

openai.api_key = '​​sk-VvF4zQ' # 在这里输入你的OpenAI API Token

os.environ["OPENAI_API_KEY"] = openai.api_key

In [ ]:
smart_system_prompt = """You are ChatGPT, a large language model trained by OpenAI.
Knowledge cutoff: 2021-09
Current date: 2023-03-15"""

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [ ]:
chat = ChatOpenAI(temperature=0)

In [ ]:
# prefix_prompt = """
# 根据这个popular的github的repo，为我总结这个github repo的功能和创新点:
# """

# content_prompt = prefix_prompt + '```\n' + txt_show + '\n```'

# smart_system_prompt_utf8 = smart_system_prompt.encode('utf-8')
# content_utf8 = content_prompt.encode('utf-8')


# messages = [
#     SystemMessage(content=smart_system_prompt_utf8),
#     HumanMessage(content=content_utf8),
# ]

# response = chat(messages)

# print(response)

In [ ]:
def get_summary( txt_show ):
    prefix_prompt = "根据这个popular的github的repo，为我总结这个github repo的功能和创新点:\n"

    content_prompt = prefix_prompt + '```\n' + txt_show + '\n```'

    smart_system_prompt_utf8 = smart_system_prompt.encode('utf-8')
    content_utf8 = content_prompt.encode('utf-8')

    messages = [
        SystemMessage(content=smart_system_prompt_utf8),
        HumanMessage(content=content_utf8),
    ]

    response = chat(messages)
    return response


In [ ]:
print(len(save_names))

354


In [ ]:
!mkdir /content/summary

In [ ]:
print(os.path.basename(save_names[0]))


openai_openai-cookbook.md


In [ ]:
import codecs
import os
import zipfile

save_path = '/content/summary'

if not os.path.exists(save_path):
    os.mkdir(save_path)

max_token = 2000

for star_num, url, save_name in zip(star_nums, github_urls, save_names):

    if not os.path.exists(save_name):
        print('warning unfound, ', save_name)
        continue

    # 读取文件
    try:
        with codecs.open(save_name, 'r', 'utf-8') as f:
            lines = f.read().split('\n')
    except:
        print('warning encode error on ', save_name)
        continue

    txt_show = ''

    total_len = 0

    for line in lines:

        current_len = len(enc.encode(line + '\n'))
        if total_len + current_len < max_token:
            txt_show += line + '\n'
            total_len += current_len
        else:
            break

    summary = get_summary(txt_show)
    # summary = 'random_string'

    base_name = os.path.basename(save_name)
    sum_file = os.path.join(save_path, base_name)
    # 保存summary到指定路径
    with open(sum_file, 'w', encoding='utf-8') as f:
        f.write(summary)

    # print('save ', sum_file)
    # break

# 将summary目录打包成zip文件
zip_file = zipfile.ZipFile('/content/zip_all.zip', 'w')
for folder, subfolders, files in os.walk(save_path):
    for file in files:
        zip_file.write(os.path.join(folder, file),
                       os.path.relpath(os.path.join(folder,file), save_path),
                       compress_type = zipfile.ZIP_DEFLATED)
zip_file.close()

warning encode error on  /content/output/microsoft_azure-openai-in-a-day-workshop.md



修改这段代码，在头部增加一个save_path = '/content/summary' 的文件夹

我希望把summary以utf8编码形式 写入到这个文件夹中，文件名使用与save_name一致的文件名

并最终把/content/summary文件夹中的所有文件打包到/content/zip_all.zip

In [ ]:
folder_names = ['tangshiye','murongfu','liyunlong',
                'Luna','wangduoyu','Ron','jiumozhi',
                'Snape','haruhi','Malfoy','xuzhu',
                'xiaofeng','duanyu','Hermione','Dumbledore',
                'wangyuyan',
                'Harry','McGonagall' ,
                'baizhantang','tongxiangyu','guofurong',
                'wanderer','zhongli','hutao',
                'Sheldon','Raj','Penny',
                'weixiaobao','qiaofeng','ayaka',
                'raidenShogun','yuqian']

character_names = ['汤师爷','慕容复','李云龙',
              'Luna','王多鱼','Ron','鸠摩智',
              'Snape','春日','Malfoy','虚竹',
              '萧峰','段誉','Hermione','Dumbledore',
              '王语嫣',
              'Harry','McGonagall',
              '白展堂','佟湘玉','郭芙蓉',
              '旅行者','钟离','胡桃',
              'Sheldon','Raj','Penny',
              '韦小宝','乔峰','神里绫华',
              '雷电将军','于谦']
name_dic = {}
for cn_role_name, en_role_name in zip(character_names, folder_names):
    name_dic.update({cn_role_name: en_role_name})

print(len(folder_names))

name_dic.update({"八重神子": "yaemiko"})

name_dic.update({"赫敏": "Hermione"})
name_dic.update({"罗恩": "Ron"})
name_dic.update({"哈利": "Harry"})
name_dic.update({"流浪者": "wanderer"})
name_dic.update({"邓布利多": "Dumbledore"})

name_dic.update({"凉宫春日": "haruhi"})
name_dic.update({"涼宮ハルヒ": "haruhi"})
name_dic.update({"涼宮": "haruhi"})


name_dic.update({"展堂": "baizhantang"})
name_dic.update({"Professor McGonagall": "McGonagall"})

for folder_name in folder_names:
    tmp_list = []
    print(folder_name)
    for key in name_dic.keys():
        if name_dic[key] == folder_name:
            # print(name_dic[key] , key )
            tmp_list.append(key)
    print(tmp_list)

32
tangshiye
['汤师爷']
murongfu
['慕容复']
liyunlong
['李云龙']
Luna
['Luna']
wangduoyu
['王多鱼']
Ron
['Ron', '罗恩']
jiumozhi
['鸠摩智']
Snape
['Snape']
haruhi
['春日', '凉宫春日', '涼宮ハルヒ', '涼宮']
Malfoy
['Malfoy']
xuzhu
['虚竹']
xiaofeng
['萧峰']
duanyu
['段誉']
Hermione
['Hermione', '赫敏']
Dumbledore
['Dumbledore', '邓布利多']
wangyuyan
['王语嫣']
Harry
['Harry', '哈利']
McGonagall
['McGonagall', 'Professor McGonagall']
baizhantang
['白展堂', '展堂']
tongxiangyu
['佟湘玉']
guofurong
['郭芙蓉']
wanderer
['旅行者', '流浪者']
zhongli
['钟离']
hutao
['胡桃']
Sheldon
['Sheldon']
Raj
['Raj']
Penny
['Penny']
weixiaobao
['韦小宝']
qiaofeng
['乔峰']
ayaka
['神里绫华']
raidenShogun
['雷电将军']
yuqian
['于谦']


tangshiye
['汤师爷']
murongfu
['慕容复']
liyunlong
['李云龙']
Luna
['Luna']
wangduoyu
['王多鱼']
Ron
['Ron', '罗恩']
jiumozhi
['鸠摩智']
Snape
['Snape']
haruhi
['春日']
Malfoy
['Malfoy']
xuzhu
['虚竹']
xiaofeng
['萧峰']
duanyu
['段誉']
Hermione
['Hermione', '赫敏']
Dumbledore
['Dumbledore', '邓布利多']
wangyuyan
['王语嫣']
Harry
['Harry', '哈利']
McGonagall
['McGonagall', 'Professor McGonagall']
baizhantang
['白展堂', '展堂']
tongxiangyu
['佟湘玉']
guofurong
['郭芙蓉']
wanderer
['旅行者', '流浪者']
zhongli
['钟离']
hutao
['胡桃']
Sheldon
['Sheldon']
Raj
['Raj']
Penny
['Penny']
weixiaobao
['韦小宝']
qiaofeng
['乔峰']
ayaka
['神里绫华']
raidenShogun
['雷电将军']
yuqian
['于谦']


In [9]:
print(len(star_nums))
print(len(github_urls))
print(len(save_names))

354
354
297


In [10]:
!unzip /content/zip_all.zip


Archive:  /content/zip_all.zip
  inflating: itamargol_openai.md     
  inflating: bentoml_OpenLLM.md      
  inflating: xuwenhao_mactalk-ai-course.md  
  inflating: conceptofmind_toolformer.md  
  inflating: MedalCollector_Orator.md  
  inflating: irgolic_AutoPR.md       
  inflating: jondurbin_airoboros.md  
  inflating: andylokandy_gpt-4-search.md  
  inflating: alexanderatallah_window.ai.md  
  inflating: nod-ai_SHARK.md         
  inflating: MiuLab_Taiwan-LLaMa.md  
  inflating: jina-ai_dev-gpt.md      
  inflating: intel_intel-extension-for-transformers.md  
  inflating: 3Alan_DocsMind.md       
  inflating: openai_openai-cookbook.md  
  inflating: daveebbelaar_langchain-experiments.md  
  inflating: juncongmoo_chatllama.md  
  inflating: kaarthik108_snowChat.md  
  inflating: jbrukh_gpt-jargon.md    
  inflating: hnawaz007_pythondataanalysis.md  
  inflating: CambioML_pykoi.md       
  inflating: amosjyng_langchain-visualizer.md  
  inflating: Jaseci-Labs_jaseci.md   
  inflating

In [13]:
import os
import re

import wget
from tqdm import tqdm

output_dir = '/content/output'

save_names = []

count = 0

url2summary = {}

for i in range(len(github_urls)):

    url = github_urls[i]

    count = count + 1

    repo_name = re.search(r'github.com/(.+)', url).group(1)

    readme_filenames = ['README.md', 'Readme.md', 'readme.md']

    # downloaded = False

    for filename in readme_filenames:

        readme_url = f'https://raw.githubusercontent.com/{repo_name}/main/{filename}'

        save_name = repo_name.replace('/', '_') + '.md'

        possible_summary = f'/content/{save_name}'

        if os.path.exists(os.path.join(output_dir, possible_summary)):
            url2summary[url] = possible_summary
            # downloaded = True
            break

    # readme_url = f'https://raw.githubusercontent.com/{repo_name}/main/README.md'

    # save_name = repo_name.replace('/', '_')

    # print(f'Downloaded {outfile}')

    # outfile = os.path.join(output_dir, save_name + '.md')

    # wget.download(readme_url, out=outfile)

print(url2summary)



{'https://github.com/openai/openai-cookbook': '/content/openai_openai-cookbook.md', 'https://github.com/AntonOsika/gpt-engineer': '/content/AntonOsika_gpt-engineer.md', 'https://github.com/imartinez/privateGPT': '/content/imartinez_privateGPT.md', 'https://github.com/LAION-AI/Open-Assistant': '/content/LAION-AI_Open-Assistant.md', 'https://github.com/microsoft/TaskMatrix': '/content/microsoft_TaskMatrix.md', 'https://github.com/hpcaitech/ColossalAI': '/content/hpcaitech_ColossalAI.md', 'https://github.com/reworkd/AgentGPT': '/content/reworkd_AgentGPT.md', 'https://github.com/OpenBB-finance/OpenBBTerminal': '/content/OpenBB-finance_OpenBBTerminal.md', 'https://github.com/geekan/MetaGPT': '/content/geekan_MetaGPT.md', 'https://github.com/jerryjliu/llama_index': '/content/jerryjliu_llama_index.md', 'https://github.com/StanGirard/quivr': '/content/StanGirard_quivr.md', 'https://github.com/openai/chatgpt-retrieval-plugin': '/content/openai_chatgpt-retrieval-plugin.md', 'https://github.com/P

In [16]:
for i in range(len(github_urls)):

    url = github_urls[i]

    star_num = star_nums[i]

    summary = ''

    repo_name = re.search(r'github.com/(.+)', url).group(1)

    if url in url2summary.keys():

        save_name = url2summary[url]

        # read save_name in utf-8 code
        with open(save_name, 'r', encoding='utf-8') as f:
            summary = f.read()

    print('---\n')
    print(f'{star_num} [{repo_name}]({url})\n')
    print(summary)
    break

---

46276 [openai/openai-cookbook](https://github.com/openai/openai-cookbook)

这个GitHub仓库是OpenAI Cookbook，它分享了使用OpenAI API完成常见任务的示例代码。

该仓库的功能和创新点包括：

1. 提供了使用OpenAI API的示例代码，涵盖了多个领域和应用，如API使用、GPT模型、嵌入、Fine-tuning GPT-3、DALL-E、Whisper等。
2. 提供了针对不同任务和应用的指南和示例，帮助用户理解和使用OpenAI API。
3. 示例代码大部分使用Python编写，但其中的概念可以应用于任何编程语言。
4. 最近添加/更新的部分列出了最新的示例代码，包括如何微调聊天模型、如何评估抽象摘要、如何使用搜索API和重新排序进行问答等。
5. 提供了与OpenAI API相关的资源链接，包括OpenAI Playground、OpenAI文档、OpenAI帮助中心、OpenAI社区论坛等，帮助用户更好地了解和使用OpenAI API。
6. 在"Related resources from around the web"部分，列出了一些与GPT相关的工具和论文，包括用于提示生成的库和工具、提示工程指南等。

总之，OpenAI Cookbook是一个开源的GitHub仓库，提供了使用OpenAI API的示例代码和指南，帮助用户在各种任务和应用中使用OpenAI API，并提供了与GPT相关的其他资源链接。


In [21]:
with open('/content/final_output.md', 'w', encoding='utf-8') as f:

    for i in range(len(github_urls)):

        url = github_urls[i]

        star_num = star_nums[i]

        summary = ''

        repo_name = re.search(r'github.com/(.+)', url).group(1)

        if url in url2summary.keys():

            save_name = url2summary[url]

            # read save_name in utf-8 code
            with open(save_name, 'r', encoding='utf-8') as f_current:
                summary = f_current.read()

        f.write('\n---\n')
        f.write(f'\n{star_num} [{repo_name}]({url})\n')
        f.write(summary)